In [1]:
import config
import requests
import praw
import pandas as pd
import pprint

In [2]:
user_agent = "python:ministryofedCSC_redditscraper:v0.0.1 (by rahm)"

In [3]:
reddit = praw.Reddit(client_id=config.client_id,
                     client_secret=config.client_secret,
                     user_agent=user_agent,
                    )

In [4]:
def extractSubmissionData(rs):
    """
    rs: being a reddit submission type
    return: list of attributes from submission
    """
    
    author_name = "[deleted]" if rs.author is None else rs.author.name
    
    return [rs.name, rs.title, rs.subreddit.display_name, author_name,\
            rs.score, rs.upvote_ratio, rs.num_comments, rs.selftext,\
            rs.url, rs.created_utc]

In [5]:
submissionsCol = ["id", "title", "subreddit", "author",\
                 "score", "upvote_ratio", "num_comments",\
                 "selftext", "url", "created_utc"]
totalSubmissionsData = pd.DataFrame([], columns=submissionsCol)


### Get submissions from Reddit

In [14]:



# query parameters
subToSearch = 'ontario'
sort = 'new'
time_filter = 'all'
# limit how many reddit posts you want
limit = 1010
# what to search on
#query = 'title:education'
query = ''
lastSubmission = None

submissionsData = pd.DataFrame([], columns=submissionsCol)

i = 0
while limit > 0:
    if limit > 1000:
        # max limit is 100, so if larger, set a throttled limit
        # and then save the difference
        thrLimit = 1000
        limit -= 1000
    else:
        thrLimit = limit
        limit -= 1000
    
    # additional query params that get passed to an internal class
    # 'praw.models.ListingGenerator'
    queryParams = {}
    if lastSubmission:
        queryParams.update({"after": lastSubmission}) #if lastSubmission != None else pass
    
    if query is None or query == '':
        submissions = reddit.subreddit(subToSearch).new(limit=thrLimit, params=queryParams)
        
    else:
        submissions = reddit.subreddit(subToSearch)\
                    .search(query, sort=sort, time_filter=time_filter,\
                           limit=thrLimit, params=queryParams)
    for sub in submissions:
        print(sub.id)
        print(sub.title)
        attrs = extractSubmissionData(sub)
        submissionsData = submissionsData.concat(attrs, axis=1)
        # we can use this last submission id as a starting point for our next batch of submissions
        lastSubmission = sub.name
        i+= 1
        print(i)
# return submissionsData
        

totalSubmissionsData = pd.concat(totalSubmissionsData, submissionsData)

print(totalSubmissionsData.head())
print(len(totalSubmissionsData))

ebkm10
Ontario Looking For Innovative Ideas on How to Help Reduce Poverty


AttributeError: 'DataFrame' object has no attribute 'concat'

In [12]:
submissionsData.head()

,0,author,created_utc,id,num_comments,score,selftext,subreddit,title,upvote_ratio,url
0,t3_ebkm10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ontario Looking For Innovative Ideas on How to...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DanFromDorval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
commentsCol = ["id", "subreddit", "submission_id", "author",\
              "body", "score", "created_utc", "conversation_id"]
conversationCol = ["id", "top_parent_id", "direct_parent_id",\
                  "child_comment_id"]


In [61]:
from collections import deque

def getReplyData(cf, top_level_id, convo_id):
    """
    comment: commentForest Object
    top_level_id: id of the top level comment
    return: tuple of list of lists: (lista, listb), where lista
            is a list of commentData, and listb is a list of convoData
    """
    cf.replace_more(limit=None)
    commentData = []
    convoData = []
    
    for com in list(cf):
        commentData.append([com.name, com.subreddit.display_name, com.submission.id,\
                            com.author.name, com.body, com.score,\
                            convo_id, com.created_utc])
        
        convoData.append([convo_id, top_level_id, com.parent_id, com.id])
        
        # according to documentation, need to call refresh before accessing reply info
        com.refresh()
        
        replyCommentData, replyConvoData = getReplyData(com.replies, top_level_id, convo_id)
        
        commentData += replyCommentData
        convoData += replyConvoData
        
    return (commentData, convoData)
    
def getCommentsFromSubmission(subm_id):
    """
    subm_id: submission id
    Return: list of comment objects from given submisison
    """
    commentData = []
    convoData = []
    # we do this indexing subm_id[2:] because subm_id is
    # the full id of a submission, of the form t2_xxxxx"
    # this reddit.submission method doesnt require the prefix t2_
    subm = reddit.submission(id=subm_id[3:])
    print(subm.title)
    
    # get rid of 'MoreComments' instances
    subm.comments.replace_more(limit=None)
    # top level comments added to deque datastructure
    
    #comments = deque(subm.comments[:])
    comments = subm.comments


    convo_id = 0

#     while len(comments) > 0:
    for com in comments:
        #com = comments.pop
        commentData.append([com.name, com.subreddit.display_name, subm_id,\
                com.author.name, com.body, com.score,\
                convo_id, com.created_utc])
        
        convoData.append([convo_id, None, None, com.id])
        
#         if top_level_id == None:
#             conversation_data = \[convo_id, ]

#         top_level_id = com.name if top_level_id != None
        top_level_id = com.name
        
        com.refresh()
        replyCommentData, replyConvoData = getReplyData(com.replies, top_level_id, convo_id)
        
        commentData += replyCommentData
        convoData += replyConvoData
      
        convo_id += 1
    
    return (commentData, convoData)

In [62]:
pprint.pprint(getCommentsFromSubmission('t3_e9hsir'))

Ontario staring down a $7-billion bill for education talks: Lecce
([['t1_faj3e53',
   'ontario',
   't3_e9hsir',
   'TwitchyJC',
   'The number goes up every time. \n'
   '\n'
   '&#x200B;\n'
   '\n'
   'Also I love the bit at the end about reaching an agreement with EWAO. '
   "Lecce mentions it's an example of him being reasonable at the bargaining "
   'table. What a laughable statement.',
   8,
   0,
   1576123352.0],
  ['t1_faj09kn',
   'ontario',
   't3_e9hsir',
   'CHoDub',
   'I hope you know this is an absolutely false claim... even based on '
   'exaggeration.',
   13,
   1,
   1576120965.0],
  ['t1_faj0n7o',
   'ontario',
   't3_e9hsir',
   'northernontario1',
   'lecce is turning this into a farce',
   9,
   2,
   1576121247.0],
  ['t1_fajxd5q',
   'ontario',
   'e9hsir',
   'ILikeStyx',
   '> lecce is ~~turning this into~~ a farce',
   5,
   2,
   1576156700.0],
  ['t1_fajzlih',
   'ontario',
   'e9hsir',
   'CHoDub',
   "To be honest, I think he's doing a great job.  He w

In [64]:
# create empty df
commentsDataDf = pd.DataFrame([], columns = commentsCol)
conversationDataDf = pd.DataFrame([], columns = conversationCol)

for i, row in submissionsDf.iterrows():
    subm_id = row.id
    comments, convo = getCommentsFromSubmission(subm_id)
    
    commentsDataDf = commentsDataDf.append(comments)
    conversationDataDf = conversationDataDf.append(convo)
    break

Ontario staring down a $7-billion bill for education talks: Lecce


In [65]:
print(len(commentsDataDf))
print(len(conversationDataDf))

16
16
